In [4]:
import pandas as p

In [10]:
df = p.read_csv("gs://vertexaioutputs/Historical Financial Data With ER and AR Calulated Updated.csv")
df = df[(df['Time'] > '2007-01-01') & (df['Time'] < '2022-08-31')]
rfr = p.read_csv('gs://vertexaioutputs/Stock Table.csv')
rfr.columns = ['ticker', 'Time', 'vw', 'rfr1m', 'rfr3m', 'rfr6m', 'rfr1y', 'rfr5y']
rfr = rfr[['Time', 'rfr1m', 'rfr3m', 'rfr6m', 'rfr1y', 'rfr5y']]
rfr = rfr.drop_duplicates(subset = "Time")

KeyError: 'Time'

In [3]:
penis = p.merge(df, rfr, on='Time')
del df

In [4]:
#[['BETA_SEC1M', 'BETA_MRKT1M', 'ER_MRKT1M', 'ER_SEC1M', 'RFR1M']]
DF = penis[['Ticker', 'Sector', 'Time', 'ER_1M',
       'AR_1M', 'ER_3M', 'AR_3M', 'ER_6M', 'AR_6M', 'ER_1Y', 'AR_1Y', 'ER_5Y',
       'AR_5Y', 'rfr1m', 'rfr3m', 'rfr6m', 'rfr1y', 'rfr5y']]
del penis

print(len(DF))

16117712


In [5]:
zz = ['ER_MRKT1M', 'ER_MRKT3M', 'ER_MRKT6M', 'ER_MRKT1Y', 'ER_MRKT5Y']
yy = ['ER_SEC1M', 'ER_SEC3M', 'ER_SEC6M', 'ER_SEC1Y', 'ER_SEC5Y']
secs = ['Basic Materials', 'Communication Services', 'Consumer Cyclical',
       'Consumer Defensive', 'Energy', 'Financial Services', 'Healthcare',
       'Industrial', 'Real Estate', 'Sector Unknown', 'Technology',
       'Utilities']

aa = ['ER_1M','ER_3M', 'ER_6M', 'ER_1Y','ER_5Y']

datas = p.DataFrame()
for i, j in zip(aa, zz):
    a = DF.groupby(['Time'])[i].sum().reset_index()
    datas[j] = a[i]
datas['Time'] = a['Time']

DF = p.merge(DF, datas, on='Time', how = 'left')
del datas   
del a

# Getting Expected Return of the Sector

In [6]:
def calc_ERS(data,tf):
    ERS = []
    new = data.groupby(['Time', 'Sector'])[tf].sum().reset_index()
    for i,j in zip(data['Time'], data['Sector']):
        ERS.append(float(new.loc[(new['Sector'] == j) & (new['Time'] == i)][tf]))
    print(j)
    return ERS


In [7]:
DF = DF.head(2000000)

DF['ER_SEC1M'] = calc_ERS(DF,'ER_1M')
print('Done with 1M')
DF['ER_SEC3M'] = calc_ERS(DF,'ER_3M')
print('Done with 3M')
DF['ER_SEC6M'] = calc_ERS(DF,'ER_6M')
print('Done with 6M')
DF['ER_SEC1Y'] = calc_ERS(DF,'ER_1Y')
print('Done with 1Y')
DF['ER_SEC5Y'] = calc_ERS(DF,'ER_5Y')
print('Done with 5Y')

Sector Unknown
Done with 1M
Sector Unknown
Done with 3M
Sector Unknown
Done with 6M
Sector Unknown
Done with 1Y
Sector Unknown
Done with 5Y


In [8]:
DF.to_csv('gs://vertexaioutputs/Historical Financial Data With ER and AR Calulated for Market and Sectors Updated.csv', index = False)

In [5]:
# Beta = (Expected Return of the Stock - RFR)/(Expected Return of the market - RFR)

DF = p.read_csv('gs://vertexaioutputs/Historical Financial Data With ER and AR Calulated for Market and Sectors Updated.csv')
df = p.DataFrame()

df['Time'] = DF['Time']
df['Ticker'] = DF['Ticker']
df['Sector'] = DF['Sector']

df['BETA_SEC1M'] = (DF['ER_1M']-DF['rfr1m'])/(DF['ER_SEC1M']-DF['rfr1m'])
df['BETA_MRKT1M'] = (DF['ER_1M']-DF['rfr1m'])/(DF['ER_MRKT1M']-DF['rfr1m'])

df['BETA_SEC3M'] = (DF['ER_3M']-DF['rfr3m'])/(DF['ER_SEC1M']-DF['rfr3m'])
df['BETA_MRKT3M'] = (DF['ER_3M']-DF['rfr3m'])/(DF['ER_MRKT1M']-DF['rfr3m'])

df['BETA_SEC6M'] = (DF['ER_6M']-DF['rfr6m'])/(DF['ER_SEC1M']-DF['rfr6m'])
df['BETA_MRKT6M'] = (DF['ER_6M']-DF['rfr6m'])/(DF['ER_MRKT1M']-DF['rfr6m'])

df['BETA_SEC1Y'] = (DF['ER_1Y']-DF['rfr1y'])/(DF['ER_SEC1M']-DF['rfr1y'])
df['BETA_MRKT1Y'] = (DF['ER_1Y']-DF['rfr1y'])/(DF['ER_MRKT1M']-DF['rfr1y'])

df['BETA_SEC5Y'] = (DF['ER_5Y']-DF['rfr5y'])/(DF['ER_SEC1M']-DF['rfr5y'])
df['BETA_MRKT5Y'] = (DF['ER_5Y']-DF['rfr5y'])/(DF['ER_MRKT1M']-DF['rfr5y'])

df['RFR1M'] = DF['rfr1m']
df['RFR3M'] = DF['rfr1m']
df['RFR6M'] = DF['rfr1m']
df['RFR1Y'] = DF['rfr1m']
df['RFR5Y'] = DF['rfr1m']

df['ER_SEC1M'] = DF[['ER_SEC1M']]
df['ER_SEC3M'] = DF[['ER_SEC3M']]
df['ER_SEC6M'] = DF[['ER_SEC6M']]
df['ER_SEC1Y'] = DF[['ER_SEC1Y']]
df['ER_SEC5Y'] = DF[['ER_SEC5Y']]

df['ER_MRKT1M'] = DF[['ER_MRKT1M']]
df['ER_MRKT3M'] = DF[['ER_MRKT3M']]
df['ER_MRKT6M'] = DF[['ER_MRKT6M']]
df['ER_MRKT1Y'] = DF[['ER_MRKT1Y']]
df['ER_MRKT5Y'] = DF[['ER_MRKT5Y']]

df['AR_1M'] = DF['AR_1M']
df['AR_3M'] = DF['AR_3M']
df['AR_6M'] = DF['AR_6M']
df['AR_1Y'] = DF['AR_1Y']
df['AR_5Y'] = DF['AR_5Y']

df['ER_1M'] = DF['ER_1M']
df['ER_3M'] = DF['ER_3M']
df['ER_6M'] = DF['ER_6M']
df['ER_1Y'] = DF['ER_1Y']
df['ER_5Y'] = DF['ER_5Y']



df.to_csv('gs://vertexaioutputs/Training Set Two Million Updated.csv', index = False)